In [41]:
import pandas as pd

In [42]:
# 读取原数据集
df = pd.read_csv(r'E:\VSCODE\毕业论文\数据集\dazhongdianping.csv',encoding='utf-8')
# 对原数据进行处理（去除空值）
df.dropna(axis=0, how='any', inplace=True)

In [43]:
# 查看是否还有空值
df.isnull().sum()

userId                0
restId                0
rating                0
rating_price          0
rating_flavor         0
rating_waitingtime    0
comment               0
dtype: int64

### 利用Pandas库中的sample抽取指定的样本

DataFrame.sample(n=None, frac=None, replace=False, weights=None, random_state=None, axis=None)

- n是要抽取的行数。（例如n=20000时，抽取其中的2W行）

- frac 是抽取的比列。（有一些时候，我们并对具体抽取的行数不关心，我们想抽取其中的百分比，这个时候就可以选择使用frac，例如frac=0.8，就是抽取其中80%）

- replace：是否为有放回抽样，取replace=True时为有放回抽样。

- weights 这个是每个样本的权重。

- random_state 随机种子。

- axis 是选择抽取数据的行还是列。axis=0的时是抽取行，axis=1时是抽取列（也就是说axis=1时，在列中随机抽取n列，在axis=0时，在行中随机抽取n行）

In [44]:
# 利用Pandas库中的sample 抽取1%的数据
df_sample = df.sample(frac=0.01)

In [45]:
df_sample.head()

,userId,restId,rating,rating_price,rating_flavor,rating_waitingtime,comment
1010163,57769.0,11121.0,4.0,3.0,4.0,4.0,喜欢他们家的盒子，很卡通的\n偶尔去吃吃还可以，但是如果经常去的话就会比较腻了\n比较喜欢吃...
2795559,152757.0,12904.0,4.0,2.0,2.0,2.0,"这个应该也算是老字号的食品店吧,人气也是很足的,买服务态度也满好的,喜欢里面的蝴蝶酥还有鲜肉..."
2534449,138749.0,16014.0,5.0,2.0,4.0,4.0,油爆虾很入味，汁很多，用醋，绍酒，糖，酱油之类的调制。45一盘，朋友说比以前份量少了，可是看...
180702,6187.0,103198.0,4.0,1.0,3.0,2.0,菜市口脏麻V5！环境巨脏巨次，吃一次拉一次，但是还总想着去吃，哈哈哈……下雨天也排大队什么的...
1235121,65763.0,188530.0,5.0,5.0,5.0,5.0,有空就去宜家，出了找创意，还喜欢和老公买这里的雪糕和肉丸，卖场出口处，超便宜，口味也很好，可...


In [46]:
# 数据统计
df_sample[['rating','rating_price','rating_flavor','rating_waitingtime']].describe().T

,count,mean,std,min,25%,50%,75%,max
rating,26831.0,3.592673,0.867419,1.0,3.0,4.0,4.0,5.0
rating_price,26831.0,2.794193,0.922723,1.0,2.0,3.0,3.0,5.0
rating_flavor,26831.0,3.087063,0.948479,1.0,2.0,3.0,4.0,5.0
rating_waitingtime,26831.0,2.762402,0.941650,1.0,2.0,3.0,3.0,5.0


In [47]:
import jieba
import re

In [48]:
# 载入停用词库
stopwords_path = r'E:\VSCODE\毕业论文\数据集\stopwords-cn.txt'
stopwords = open(stopwords_path,encoding='utf-8').read().splitlines()

In [49]:
# 去除中文以外的所有字符
str_out = []
for cn_wrod in df_sample['comment']:
    text0 =cn_wrod  # 获取文本内容
    text1 = re.sub("[\.\!\/_,$%^*(+\"\']+|[+——！，。？、~@#￥%……&*（）]+", "", text0)  # 去除标点及特殊符号
    text2 = re.sub("[^\u4e00-\u9fa5]", "", text1)  # 去除所有非汉字内容（英文数字）
    str_out.append(text2)
str_out[0][0]

'喜'

In [50]:
# 用结巴分词并使用停用词
words_out=[]
for word in str_out:
    jieba_cut = jieba.cut(word,cut_all=False)
    jieba_word = ' '.join(list(jieba_cut))
    # str_stop_word = [word for word in jieba_cut if word not in stopwords]
    
    words_out.append(jieba_word)
words_out[0]

'喜欢 他们 家 的 盒子 很 卡通 的 偶尔 去 吃 吃 还 可以 但是 如果 经常 去 的话 就 会 比较 腻 了 比较 喜欢 吃 香草 的 一股 香香的 味道 感觉 很 温暖'

In [51]:
# 停用词
str_stop = []
for word in words_out:
    if word not in stopwords:
        str_stop.append(word)
str_stop[0]

'喜欢 他们 家 的 盒子 很 卡通 的 偶尔 去 吃 吃 还 可以 但是 如果 经常 去 的话 就 会 比较 腻 了 比较 喜欢 吃 香草 的 一股 香香的 味道 感觉 很 温暖'

### 词向量化

### 1.glove方法

In [52]:
# 导入glove模块
from glove import Glove,Corpus

In [53]:
# 准备要向量化的数据

sentense = [str_stop]
# 实例化语料库模型
corpus_model = Corpus()

In [54]:
# 训练corpus_model
corpus_model.fit(sentense,window=10)

In [55]:
corpus_model.save('corpus_model_vector')
print('Dict size: %s' % len(corpus_model.dictionary)) # 打印训练模型大小
print('Collocations: %s' % corpus_model.matrix.nnz)

Dict size: 26815
Collocations: 268255


In [56]:
# glove 的训练
glove = Glove(no_components=100,learning_rate=0.05)
glove.fit(corpus_model.matrix,epochs = 200,no_threads = 6,verbose = True)
glove.add_dictionary(corpus_model.dictionary)

Performing 200 training epochs with 6 threads
Epoch 0
Epoch 1
Epoch 2
Epoch 3
Epoch 4
Epoch 5
Epoch 6
Epoch 7
Epoch 8
Epoch 9
Epoch 10
Epoch 11
Epoch 12
Epoch 13
Epoch 14
Epoch 15
Epoch 16
Epoch 17
Epoch 18
Epoch 19
Epoch 20
Epoch 21
Epoch 22
Epoch 23
Epoch 24
Epoch 25
Epoch 26
Epoch 27
Epoch 28
Epoch 29
Epoch 30
Epoch 31
Epoch 32
Epoch 33
Epoch 34
Epoch 35
Epoch 36
Epoch 37
Epoch 38
Epoch 39
Epoch 40
Epoch 41
Epoch 42
Epoch 43
Epoch 44
Epoch 45
Epoch 46
Epoch 47
Epoch 48
Epoch 49
Epoch 50
Epoch 51
Epoch 52
Epoch 53
Epoch 54
Epoch 55
Epoch 56
Epoch 57
Epoch 58
Epoch 59
Epoch 60
Epoch 61
Epoch 62
Epoch 63
Epoch 64
Epoch 65
Epoch 66
Epoch 67
Epoch 68
Epoch 69
Epoch 70
Epoch 71
Epoch 72
Epoch 73
Epoch 74
Epoch 75
Epoch 76
Epoch 77
Epoch 78
Epoch 79
Epoch 80
Epoch 81
Epoch 82
Epoch 83
Epoch 84
Epoch 85
Epoch 86
Epoch 87
Epoch 88
Epoch 89
Epoch 90
Epoch 91
Epoch 92
Epoch 93
Epoch 94
Epoch 95
Epoch 96
Epoch 97
Epoch 98
Epoch 99
Epoch 100
Epoch 101
Epoch 102
Epoch 103
Epoch 104
Epoch 105
Epoc

In [57]:
# 保存Glove模型
glove.save('glove_model_sentence')

In [58]:
# 词向量（数组类型）
words_vectors = glove.word_vectors

In [59]:
str_stop[0]

'喜欢 他们 家 的 盒子 很 卡通 的 偶尔 去 吃 吃 还 可以 但是 如果 经常 去 的话 就 会 比较 腻 了 比较 喜欢 吃 香草 的 一股 香香的 味道 感觉 很 温暖'

In [60]:
# 查看指定词向量(列表、元组，字典不可哈希)
glove.word_vectors[glove.dictionary[str_stop[0]]]

array([-0.00042637,  0.00260173,  0.00434603,  0.00477955, -0.00234784,
        0.00171945, -0.00355883,  0.00318331,  0.00525106,  0.00256926,
       -0.00559164,  0.00129618, -0.00356281, -0.0013275 ,  0.00466399,
       -0.00509879, -0.00525182, -0.00362616, -0.00428413,  0.00018334,
       -0.00236972, -0.00091306,  0.00152951, -0.00041313,  0.00082575,
        0.0035143 , -0.00422022,  0.00289465,  0.00380301, -0.00218647,
       -0.00133181, -0.0005916 ,  0.00074599, -0.00178439, -0.00053483,
       -0.00127108,  0.00360612, -0.00585692,  0.00214786,  0.00194452,
        0.00059327,  0.00234804, -0.00511655, -0.00416349,  0.00336864,
        0.00371206, -0.00173705,  0.00414225,  0.00090369,  0.00472821,
       -0.00352779, -0.00552685, -0.00498403, -0.00350763, -0.00228075,
        0.00383519, -0.00395183,  0.00292717,  0.00449221,  0.00295432,
        0.00251587, -0.00203868,  0.00491539,  0.00108472, -0.00607617,
        0.00241704, -0.00012328, -0.00103504,  0.00398641,  0.00

In [61]:
# 近似词判定
glove.most_similar(str_stop[0],number=5)

[('第一次 吃 吉野家 就是 在 这里 了 和 一 起来 的 坏境 不错 我 比较 喜欢 牛肉 饭 吃 的 什么 记不清 了 还 送 了 一支 软陶 笔 喜欢',
  0.39161479186502285),
 ('年 去 的 记得 那 时候 不 叫 这个 名字 吧 一直 没有 点评 名字 都 变 了 呀 去 那里 溜冰 然后 同学 推荐 去 的 记得 第一个 菜 很 惊艳 啊 咖喱 牛腩 非常 难道 是 那个 时候 饿 了 吗 反正 记忆 中 总是 好吃 的',
  0.3668121311468867),
 ('饺子 还 不错 小菜 太小 饺子 汤 不让 自己 乘 就 睡 品种 太 少 上菜 太慢 了 这个 位置 上 的 餐厅 不是 特别 大 的 就是 这种 感觉 品质 一般 吧 不 推荐 总去 要是 逛街 累 了 就 随便 了',
  0.3554963795901843),
 ('这间 店 真是 好 出名 每次 去 都 要 等位 呢 间系 老公 仔 最 中意 噶 吃饺子 噶店 元 只 基本上 每个 人去 都 会 必点 噶 就是 距 老公 仔话 细细 个 就 吃 到 家 次次 都 会 来 宜家 吃饺子 不但 大 只 而且 足料 家 上 碗 朱红 汤 就 完美 啦 但是 我 觉得 紫菜汤 仲好 每次 出 街 如果 想 唔 到 去 边 吃 野噶话 差不多 就黎 呢 间 就是 潮楼 对面 好 方便 不过 就是 觉得 系间 左 呢 家 八珍 越开 越 多 但是 间间 都 没 呢 间 甘 原汁原味 其他 总 觉得 差少少 可能 其他 已经 发展 到 除 左 饺子 还有 粥 粉面 饭 可能 就 没甘 注重 吧 我 同 老公 仔去 通常 都 要 两份 饺子 因为 我 中意 玉米 饺子 而 距 中意 韭菜 饺子 所以 次次 都 要 吃 好多 呵呵 呵 真是 好 好吃 如果 你 想 吃饺子 还是 这件 八珍 的 好 嘻嘻',
  0.3531159986109511)]

### 人工标注情感分类数据集

In [68]:
# 抽取一半数据做人工标记
data_emotion = df_sample.sample(frac=0.5)
data_emotion.shape

(13416, 7)

In [69]:
# data_emotion.to_csv(r'E:\VSCODE\毕业论文\数据集\emotion.csv')

### 建立情感分类预测模型

In [316]:
# 读取数据
df_emotion = pd.read_csv(r'E:\VSCODE\毕业论文\数据集\emotion.csv')

In [317]:
# 查看空值
df_emotion.isnull().sum()

userId                0
restId                0
rating                0
rating_price          0
rating_flavor         0
rating_waitingtime    0
comment               0
emotion               0
dtype: int64

In [320]:
# 去除中文以外的所有字符
str_out_emotion = []
for cn_wrod in df_emotion['comment']:
    text0 =cn_wrod  # 获取文本内容
    text1 = re.sub("[\.\!\/_,$%^*(+\"\']+|[+——！，。？、~@#￥%……&*（）]+", "", text0)  # 去除标点及特殊符号
    text2 = re.sub("[^\u4e00-\u9fa5]", "", text1)  # 去除所有非汉字内容（英文数字）
    str_out_emotion.append(text2)
str_out_emotion[117]

''

In [337]:
# 处理文本（结巴分词）并载入停用词
emotion_words = []
for word in str_out_emotion:
    jieba_cut = jieba.cut(word,cut_all=False)
    jieba_word = [word for word in jieba_cut if word not in stopwords] 
    emotion_words.append(jieba_word)

In [349]:
df_emotion['comment'] = pd.Series(emotion_words)

In [351]:
# 查看空值
df_emotion.isnull().sum()

userId                 0
restId                 0
rating                 0
rating_price           0
rating_flavor          0
rating_waitingtime     0
comment               15
emotion                0
dtype: int64

In [352]:
# 去除空值
df_emotion.dropna(axis=0,how='any',inplace=True)

In [354]:
df_emotion['emotion'].shape

(13399,)

In [355]:
len(emotion_words)

13399

### 使用glove向量化

In [357]:
# 准备要向量化的数据
sentense_emotion = emotion_words
# 实例化语料库模型
corpus_model_emotion = Corpus()

In [360]:
# 训练corpus_model
corpus_model_emotion.fit(sentense_emotion,window=10)

In [361]:
# 保存语料库模型
corpus_model_emotion.save('corpus_model_emotion')
print('Dict size: %s' % len(corpus_model_emotion.dictionary)) # 打印训练模型大小
print('Collocations: %s' % corpus_model_emotion.matrix.nnz)

Dict size: 51041
Collocations: 2776844


In [362]:
# glove 的训练
glove_emotion = Glove(no_components=150,learning_rate=0.05)
glove_emotion.fit(corpus_model_emotion.matrix,epochs = 300,no_threads = 6,verbose = True)
glove_emotion.add_dictionary(corpus_model_emotion.dictionary)

Performing 300 training epochs with 6 threads
Epoch 0
Epoch 1
Epoch 2
Epoch 3
Epoch 4
Epoch 5
Epoch 6
Epoch 7
Epoch 8
Epoch 9
Epoch 10
Epoch 11
Epoch 12
Epoch 13
Epoch 14
Epoch 15
Epoch 16
Epoch 17
Epoch 18
Epoch 19
Epoch 20
Epoch 21
Epoch 22
Epoch 23
Epoch 24
Epoch 25
Epoch 26
Epoch 27
Epoch 28
Epoch 29
Epoch 30
Epoch 31
Epoch 32
Epoch 33
Epoch 34
Epoch 35
Epoch 36
Epoch 37
Epoch 38
Epoch 39
Epoch 40
Epoch 41
Epoch 42
Epoch 43
Epoch 44
Epoch 45
Epoch 46
Epoch 47
Epoch 48
Epoch 49
Epoch 50
Epoch 51
Epoch 52
Epoch 53
Epoch 54
Epoch 55
Epoch 56
Epoch 57
Epoch 58
Epoch 59
Epoch 60
Epoch 61
Epoch 62
Epoch 63
Epoch 64
Epoch 65
Epoch 66
Epoch 67
Epoch 68
Epoch 69
Epoch 70
Epoch 71
Epoch 72
Epoch 73
Epoch 74
Epoch 75
Epoch 76
Epoch 77
Epoch 78
Epoch 79
Epoch 80
Epoch 81
Epoch 82
Epoch 83
Epoch 84
Epoch 85
Epoch 86
Epoch 87
Epoch 88
Epoch 89
Epoch 90
Epoch 91
Epoch 92
Epoch 93
Epoch 94
Epoch 95
Epoch 96
Epoch 97
Epoch 98
Epoch 99
Epoch 100
Epoch 101
Epoch 102
Epoch 103
Epoch 104
Epoch 105
Epoc

In [363]:
# 保存Glove模型
glove_emotion.save('glove_model_emotion')

In [364]:
# 词向量（数组类型）
word_vectors = glove_emotion.word_vectors
word_vectors.shape

(51041, 150)

In [367]:
# 获取每个句子的长度
word_len = []
for i in emotion_words:
    word_len.append(len(i))

In [368]:
# 获取句子中每个词向量的总和(句向量)
word_vec_sum = []
for j in word_len:
    a = sum((word_vectors[i]) for i in range(j))
    word_vec_sum.append(a)

In [369]:
word_vec_sum_list = []
for n,i in enumerate(word_vec_sum):
    try:
        word_vec_sum_list.extend(i)
    except TypeError:
        print(n,i)
        word_vec_sum_list.extend(list(np.zeros(150)))
word_vec_sum = np.array(word_vec_sum_list).reshape((-1,150))

In [374]:
# 句向量
word_vec_sum.shape

(13399, 150)

In [373]:
df_emotion['emotion'].shape

(13399,)

In [375]:
# 载入朴素贝叶斯模型
from sklearn.naive_bayes import GaussianNB
from sklearn.model_selection import train_test_split

In [376]:
# 创建特征和目标
feature = word_vec_sum
target = df_emotion['emotion']

In [378]:
feature.shape,target.shape

((13399, 150), (13399,))

In [399]:
 # 拆分样本
 x_train,x_test,y_train,y_test = train_test_split(feature,target,test_size = 0.2,random_state = 2000)

In [400]:
# 实例化高斯函数
gaussian = GaussianNB()

In [401]:
gaussian.fit(x_train,y_train)

GaussianNB()

In [402]:
gaussian.score(x_test,y_test)

0.22014925373134328